### Reading CSV data with an inferred schema

In [2]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트

# 새로운 SparkSession 생성 - Spark 애플리케이션의 진입점
spark = (SparkSession
         .builder
         .appName("read-csv-data")  # 애플리케이션 이름 설정
         .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
         .config("spark.executor.memory", "512m")  # 실행자 메모리 설정
         .getOrCreate())  # SparkSession 생성 또는 기존 세션 반환

# 로그 레벨을 ERROR로 설정하여 불필요한 로그 출력 최소화
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/17 21:52:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# CSV 파일을 DataFrame으로 읽기 - 스키마 자동 추론 방식
df = (spark.read
      .format("csv")  # 파일 형식을 CSV로 지정
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .load("../data/netflix_titles.csv"))  # 파일 경로 지정

# 대안 방법
## CSV 파일에 헤더가 없는 경우

df = (spark.read
      .format("csv")
      .option("header", "false") # CSV 파일에 헤더가 없을 때 사용
      .load(  # 파일 로드"../data/netflix_titles.csv"))

In [4]:
# DataFrame의 내용을 화면에 출력
df.show()  # DataFrame 내용 출력

# 다른 출력 옵션들

# df.show(50)  # 처음 50개 행 출력
# df.show(10, truncate=False)  # 처음 10개 행을 잘림 없이 전체 출력

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

In [5]:
# 출력 schema of DataFrame
df.printSchema()  # DataFrame 스키마 구조 출력

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



### Reading CSV data with explicit schema

In [6]:
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StructType, StructField, StringType, IntegerType, DateType

# Define a Schema
schema = StructType  # 구조체 타입([
    StructField  # 구조체 필드("show_id", StringType(), True),
    StructField  # 구조체 필드("type", StringType(), True),
    StructField  # 구조체 필드("title", StringType(), True),
    StructField  # 구조체 필드("director", StringType(), True),
    StructField  # 구조체 필드("cast", StringType(), True),
    StructField  # 구조체 필드("country", StringType(), True),
    StructField  # 구조체 필드("date_added", DateType(), True),
    StructField  # 구조체 필드("release_year", IntegerType(), True),
    StructField  # 구조체 필드("rating", StringType(), True),
    StructField  # 구조체 필드("duration", StringType(), True),
    StructField  # 구조체 필드("listed_in", StringType(), True),
    StructField  # 구조체 필드("description", StringType(), True)])


In [7]:
# 읽기 CSV file into a DataFrame
df = (spark.read.format("csv")  # CSV 형식으로 데이터 읽기
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .schema(schema)
      .load(  # 파일 로드"../data/netflix_titles.csv"))

In [8]:
# 출력 contents of DataFrame
df.show()  # DataFrame 내용 출력

+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|      null|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|      null|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, T

### Common issues faced while working with CSV data

In [9]:
# 읽기 CSV file into a DataFrame
df = (spark.read.format("csv")  # CSV 형식으로 데이터 읽기 
      .option("header", "true")  # 첫 번째 행을 헤더로 사용 
      .option("nullValue", "null") 
      .option("escapeQuotes", "true") 
      .schema(schema) 
      .load(  # 파일 로드"../data/netflix_titles.csv")) 

In [10]:
# 출력 first 5 rows of DataFrame
df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|      null|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa|      null|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|      null|        2021| TV-MA| 1 Season|Crime

In [8]:
# 읽기 CSV file into a DataFrame
df = (spark.read
      .format("csv")
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .option("nullValue", "null")
      .option("emptyValues", "")
      .schema(schema)
      .load(  # 파일 로드"../data/netflix_titles.csv"))

In [9]:
# 출력 first 5 rows of DataFrame
df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|      null|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           null|Ama Qamata, Khosi...| South Africa|      null|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|      null|        2021| TV-MA| 1 Season|Crime

In [10]:
df = (spark.read.format("csv")  # CSV 형식으로 데이터 읽기
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .option("nullValue", "null")
      .option("dateFormat",  # 날짜 형식 지정 "LLLL d, y")
      .schema(schema)
      .load(  # 파일 로드"../data/netflix_titles.csv"))


In [11]:
# 출력 contents of DataFrame
df.show()  # DataFrame 내용 출력

+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|2021-09-25|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|2021-09-24|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, T

In [11]:
# Stop the Spark Session
spark.stop()  # Spark 세션 종료 - 리소스 정리